In [142]:
lines = ["32T3K 765",
         "T55J5 684",
         "KK677 28",
         "KTJJT 220",
         "QQQJA 483"]

# labels: A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, 2; highest A, lowest 2
# hand types: 
#   five of a kind: all 5 cards have the same label
#   four of a kind: four cards have the same label, one card has a different label
#   full house: three cards have the same label, remaining two cards share a different label
#   three of a kind: three cards have the same label, remaining two cards share two different labels
#   two pair: two cards share one label, another two cards share a second label, remaining card has a third label
#   one pair: two cards share one label, remaining three cards share three different labels
#   high card: all card labels are distinct

# if two hands have the same type, compare first card in each hand.
# if these cards are different, the hand with the stronger first card is considered stronger
# if the first card in each hand have the same label, compare the second card in each hand and continue

# win amount = bid x rank (weakest hand, lowest rank)

# PART 1
label_rank = {"A": 13, "K": 12, "Q": 11, "J": 10, "T": 9, "9": 8, "8": 7, "7": 6, "6": 5, "5": 4, "4": 3, "3": 2, "2": 1}
reverse_label_rank = {13: "A", 12: "K", 11: "Q", 10: "J", 9: "T", 8: "9", 7: "8", 6: "7", 5: "6", 4: "5", 3: "4", 2: "3", 1: "2"}

def get_hand(lines):
    hands = {}
    for i in range(len(lines)):
        hands[i+1] = lines[i][:5]
    return hands

def get_bids(lines):
    bids = []
    for i in range(len(lines)):
        bids.append(int(lines[i][6:]))
    return bids

def check_handtypes(hands):
    handtypes = {}
    for number, hand in hands.items():
        not_duplicate = []
        duplicates = []
        for x in range(len(hand)):
            if hand[x] not in not_duplicate:
                not_duplicate.append(hand[x])
            else:
                duplicates.append(hand[x])
        handtypes = without_joker_rule(not_duplicate, duplicates, handtypes, number)
    return handtypes

def without_joker_rule(not_duplicate, duplicates, handtypes, number):
    if len(not_duplicate) == 1 and len(duplicates) == 4:
        handtypes[number] = "five of a kind"
    elif len(not_duplicate) == 2 and len(duplicates) == 3:
        if len(set(duplicates)) == 1:
            handtypes[number] = "four of a kind"
        elif len(set(duplicates)) == 2:
            handtypes[number] = "full house"
    elif len(not_duplicate) == 3 and len(duplicates) == 2:
        if len(set(duplicates)) == 1:
            handtypes[number] = "three of a kind"
        elif len(set(duplicates)) == 2:
            handtypes[number] = "two pair"
    elif len(not_duplicate) == 4 and len(duplicates) == 1:
        handtypes[number] = "one pair"
    elif len(not_duplicate) == 5:
        handtypes[number] = "high card"
    return handtypes

def order_strength(hands, handtypes, label_rank, reverse_label_rank):
    same_handtypes = {"five of a kind": [], "four of a kind": [], "full house": [], "three of a kind": [], "two pair": [], "one pair": [], "high card": []}
    for i, handtype in handtypes.items():
        same_handtypes[handtype].append(hands[i])

    for same_handtype, j in same_handtypes.items():
        if len(j) > 1:
            cards = check_cards(j, label_rank)
            for x in range(len(cards)):
                cards[x] = "".join([reverse_label_rank[y] for y in cards[x]])
            j = cards
    
    hands_ranked = []
    for same_handtype, k in same_handtypes.items():
        if len(k) > 0:
            for z in range(len(k)):
                for hand_number, five_cards in hands.items():
                    if five_cards == k[z]:
                        hands_ranked.append(hand_number)
    return hands_ranked

    
def check_cards(cards, label_rank):
    for i in range(len(cards)):
        cards[i] = [label_rank[j] for j in cards[i]]
    n = len(cards)
    for i in range(1, n):
        j = i
        while j > 0:
            cards = comparing_cards(cards, j, 0)
            j -= 1
    return cards

def comparing_cards(cards, j, index):
    if cards[j][index] > cards[j-1][index]:
        cards[j-1], cards[j] = cards[j], cards[j-1]
    if cards[j][index] == cards[j-1][index]:
        comparing_cards(cards, j, index+1)
    return cards

def calculate_winnings(bids, hands_ranked):
    total_winnings = 0
    index = 0
    n = max(hands_ranked)
    for j in hands_ranked:
        total_winnings += (bids[j-1] * (n-index))
        index += 1
    return total_winnings

# PART 2
new_label_rank = {"A": 13, "K": 12, "Q": 11, "T": 10, "9": 9, "8": 8, "7": 7, "6": 6, "5": 5, "4": 4, "3": 3, "2": 2, "J": 1}
new_reverse_label_rank = {13: "A", 12: "K", 11: "Q", 10: "T", 9: "9", 8: "8", 7: "7", 6: "6", 5: "5", 4: "4", 3: "3", 2: "2", 1: "J"}

def new_check_handtypes(hands):
    handtypes = {}
    for number, hand in hands.items():
        num_j = 0
        not_duplicate = []
        duplicates = []
        for x in range(len(hand)):
            if hand[x] == "J":
                num_j += 1
            elif hand[x] not in not_duplicate:
                not_duplicate.append(hand[x])
            else:
                duplicates.append(hand[x])
        if num_j == 0:
            handtypes = without_joker_rule(not_duplicate, duplicates, handtypes, number)
        elif num_j == 1: #JAAAA, JAAA8, JAA88, JAA87, JA987
            if len(not_duplicate) == 1 and len(duplicates) == 3: #JAAAA
                handtypes[number] = "five of a kind"
            elif len(not_duplicate) == 2 and len(duplicates) == 2: #JAAA8 or JAA88
                if len(set(duplicates)) == 1:
                    handtypes[number] = "four of a kind"
                elif len(set(duplicates)) == 2:
                    handtypes[number] = "full house"
            elif len(not_duplicate) == 3 and len(duplicates) == 1: #JAA87
                handtypes[number] = "three of a kind"
            elif len(not_duplicate) == 4: #JA987
                handtypes[number] = "one pair"
        elif num_j == 2: #JJAAA, #JJAA8, #JJA98
            if len(not_duplicate) == 1 and len(duplicates) == 2: #JJAAA
                handtypes[number] = "five of a kind"
            elif len(not_duplicate) == 2 and len(duplicates) == 1: #JJAA8
                handtypes[number] = "four of a kind"
            elif len(not_duplicate) == 3: #JJA98
                handtypes[number] = "three of a kind"
        elif num_j == 3: #JJJAA, JJJA8
            if len(not_duplicate) == 1 and len(duplicates) == 1: #JJJAA
                handtypes[number] = "five of a kind"
            elif len(not_duplicate) == 2: #JJJA8
                handtypes[number] = "four of a kind"
        elif num_j >= 4: #JJJJA or #JJJJJ
            handtypes[number] = "five of a kind"
    return handtypes

with open ('day7.txt') as file:
    lines = file.read().split("\n")

hands = get_hand(lines)
bids = get_bids(lines)
handtypes = check_handtypes(hands)
hands_ranked = order_strength(hands, handtypes, label_rank, reverse_label_rank)
total_winnings = calculate_winnings(bids, hands_ranked)

new_handtypes = new_check_handtypes(hands)
new_hands_ranked = order_strength(hands, new_handtypes, new_label_rank, new_reverse_label_rank)
new_total_winnings = calculate_winnings(bids, new_hands_ranked)

print(f"part 1: {total_winnings}")
print(f"part 2: {new_total_winnings}")

part 1: 248396258
part 2: 246436046
